# Chamgab ML Model Retraining

PC 종료 후에도 클라우드에서 모델을 재훈련할 수 있는 노트북입니다.

## 파이프라인
1. 레포 클론 + 의존성 설치
2. Supabase에서 훈련 데이터 fetch
3. Feature Engineering (71개 피처)
4. XGBoost 학습 (Early Stopping + TimeSeriesSplit CV)
5. SHAP Explainer 생성
6. 모델 저장 (.pkl)
7. HuggingFace Spaces 배포 (선택)
8. Supabase 분석 재생성 (선택)

## 소요 시간
- 데이터 fetch: ~5분
- 학습: ~10-20분
- 분석 재생성: ~30분
- **총: ~1시간 이내**

---
## 0. 환경 설정

아래 셀에 Supabase 키를 입력하세요. Colab Secrets 사용을 권장합니다.

In [ ]:
# ============================================================
# 방법 1: Colab Secrets (권장)
# 왼쪽 사이드바 > 열쇠 아이콘 > 시크릿 추가
# ============================================================
import os

try:
    from google.colab import userdata
    os.environ['SUPABASE_URL'] = userdata.get('SUPABASE_URL')
    os.environ['SUPABASE_SERVICE_KEY'] = userdata.get('SUPABASE_SERVICE_KEY')
    # 선택: HuggingFace 배포용
    try:
        os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
    except Exception:
        pass
    print('Colab Secrets에서 키 로드 완료')
except Exception:
    pass

# ============================================================
# 방법 2: 직접 입력 (Secrets 미사용 시 주석 해제)
# ============================================================
# os.environ['SUPABASE_URL'] = 'https://YOUR_PROJECT.supabase.co'
# os.environ['SUPABASE_SERVICE_KEY'] = 'eyJ...'  # service_role key

# 확인
assert os.environ.get('SUPABASE_URL'), 'SUPABASE_URL이 설정되지 않았습니다!'
assert os.environ.get('SUPABASE_SERVICE_KEY'), 'SUPABASE_SERVICE_KEY가 설정되지 않았습니다!'
print(f'Supabase URL: {os.environ["SUPABASE_URL"][:40]}...')

---
## 1. 레포 클론 + 의존성 설치

In [ ]:
%%bash
# git-lfs 설치 (모델 .pkl 파일 다운로드에 필요)
apt-get install -qq git-lfs
git lfs install

# 레포 클론 (이미 있으면 pull)
if [ -d "chamgab" ]; then
    cd chamgab && git pull
else
    git clone https://github.com/choiwjun/chamgab.git
fi

In [ ]:
# ML API 의존성 설치
!pip install -q xgboost>=2.0.0 shap>=0.45.0 scikit-learn>=1.4.0 \
    pandas>=2.2.0 numpy>=1.26.0 supabase>=2.0.0 python-dotenv>=1.0.0 \
    httpx>=0.27.0 optuna>=3.5.0 lightgbm>=4.0.0 pyarrow>=15.0.0 \
    lxml>=5.0.0 psycopg2-binary>=2.9.0

print('\n의존성 설치 완료!')

In [ ]:
# 작업 디렉토리 설정
import os
os.chdir('/content/chamgab/ml-api')

# .env 파일 생성 (Colab 환경변수 → .env)
with open('.env', 'w') as f:
    f.write(f'SUPABASE_URL={os.environ["SUPABASE_URL"]}\n')
    f.write(f'SUPABASE_SERVICE_KEY={os.environ["SUPABASE_SERVICE_KEY"]}\n')

print(f'작업 디렉토리: {os.getcwd()}')
print(f'.env 파일 생성 완료')

---
## 2. 모델 재훈련

3가지 모드 중 선택:
- **기본**: Early Stopping + CV (~10분)
- **튜닝**: Optuna 200 trials (~30분)
- **앙상블**: XGBoost + LightGBM (~15분)

In [ ]:
# ============================================================
# 훈련 모드 선택
# ============================================================
BASIC_TRAIN = True       # 기본 학습 (~10분)
OPTUNA_TUNE = True       # 하이퍼파라미터 튜닝 (~30분)
ENSEMBLE = True          # LightGBM 앙상블 (~15분)
TUNE_TRIALS = 200        # Optuna trials 수

# 훈련 커맨드 구성
cmd = 'python -m scripts.train_model'
if OPTUNA_TUNE:
    cmd += f' --tune --trials {TUNE_TRIALS}'
if ENSEMBLE:
    cmd += ' --ensemble'

print(f'실행 커맨드: {cmd}')
print(f'예상 소요: {"~45분" if OPTUNA_TUNE and ENSEMBLE else "~30분" if OPTUNA_TUNE else "~15분" if ENSEMBLE else "~10분"}')

In [ ]:
# 모델 학습 실행
import subprocess
import sys

result = subprocess.run(
    cmd.split(),
    cwd='/content/chamgab/ml-api',
    capture_output=False,
    text=True,
    env={**os.environ, 'PYTHONPATH': '/content/chamgab/ml-api'}
)

if result.returncode == 0:
    print('\n' + '='*60)
    print('모델 학습 성공!')
    print('='*60)
else:
    print(f'\n학습 실패 (exit code: {result.returncode})')

---
## 3. 학습 결과 확인

In [ ]:
import json

metrics_path = '/content/chamgab/ml-api/app/models/apartment_model_metrics.json'

with open(metrics_path, 'r', encoding='utf-8') as f:
    metrics = json.load(f)

print('=== 학습 결과 ===')
print(f'학습 시간: {metrics["training_duration_seconds"]}초')
print(f'Train: {metrics["data_summary"]["train_samples"]}건')
print(f'Val:   {metrics["data_summary"]["val_samples"]}건')
print(f'Test:  {metrics["data_summary"]["test_samples"]}건')
print(f'Features: {metrics["data_summary"]["features"]}개')
print()
print('=== 평가 지표 ===')
m = metrics['metrics']
print(f'MAE:  {m["mae"]:,.0f} 원')
print(f'RMSE: {m["rmse"]:,.0f} 원')
print(f'R2:   {m["r2"]:.4f}')
print(f'MAPE: {m["mape"]:.2f}%')
print()
print('=== 교차 검증 ===')
cv = metrics['cross_validation']
print(f'CV MAPE: {cv["cv_mape_mean"]:.2f}% +/- {cv["cv_mape_std"]:.2f}%')
print()
print('=== Feature Importance Top 10 ===')
for i, fi in enumerate(metrics['feature_importance_top20'][:10], 1):
    bar = '#' * int(fi['importance'] * 200)
    print(f'{i:2d}. {fi["feature"]:30s} {fi["importance"]:.4f} {bar}')

In [ ]:
# 모델 파일 크기 확인
import os

models_dir = '/content/chamgab/ml-api/app/models'
pkl_files = ['xgboost_model.pkl', 'shap_explainer.pkl', 'feature_artifacts.pkl', 'residual_info.pkl']

print('=== 모델 파일 ===')
total = 0
for f in pkl_files:
    path = os.path.join(models_dir, f)
    if os.path.exists(path):
        size = os.path.getsize(path)
        total += size
        print(f'  {f:30s} {size/1024:,.1f} KB')
    else:
        print(f'  {f:30s} (없음)')
print(f'  {"합계":30s} {total/1024/1024:,.1f} MB')

---
## 4. 모델 다운로드 / HuggingFace 배포

학습된 모델을 가져가는 3가지 방법:

In [ ]:
# 방법 A: Google Drive에 저장 (가장 간단)
SAVE_TO_DRIVE = True  # False로 변경하면 스킵

if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    import shutil
    dest = '/content/drive/MyDrive/chamgab-models'
    os.makedirs(dest, exist_ok=True)

    for f in pkl_files + ['apartment_model_metrics.json']:
        src = os.path.join(models_dir, f)
        if os.path.exists(src):
            shutil.copy2(src, dest)
            print(f'  복사: {f} -> {dest}/')

    print(f'\nGoogle Drive 저장 완료: {dest}')

In [ ]:
# 방법 B: HuggingFace Spaces에 직접 배포
DEPLOY_TO_HF = False  # True로 변경하면 실행
HF_REPO = 'YOUR_USERNAME/chamgab-ml-api'  # HuggingFace Spaces repo

if DEPLOY_TO_HF:
    !pip install -q huggingface_hub

    from huggingface_hub import HfApi

    hf_token = os.environ.get('HF_TOKEN')
    assert hf_token, 'HF_TOKEN이 설정되지 않았습니다!'

    api = HfApi(token=hf_token)

    for f in pkl_files:
        local_path = os.path.join(models_dir, f)
        if os.path.exists(local_path):
            api.upload_file(
                path_or_fileobj=local_path,
                path_in_repo=f'app/models/{f}',
                repo_id=HF_REPO,
                repo_type='space',
            )
            print(f'  업로드: {f}')

    print(f'\nHuggingFace Spaces 배포 완료: {HF_REPO}')
    print('Space가 자동으로 재빌드됩니다.')

In [ ]:
# 방법 C: 직접 다운로드 (브라우저)
DOWNLOAD_FILES = False  # True로 변경하면 실행

if DOWNLOAD_FILES:
    from google.colab import files

    # ZIP으로 묶어서 다운로드
    import zipfile
    zip_path = '/content/chamgab_models.zip'
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for f in pkl_files + ['apartment_model_metrics.json']:
            src = os.path.join(models_dir, f)
            if os.path.exists(src):
                zf.write(src, f)

    files.download(zip_path)
    print('다운로드 시작!')

---
## 5. (선택) Supabase 분석 데이터 재생성

새 모델로 chamgab_analyses + price_factors를 재생성합니다.

**주의**: 전체 25,890건 재생성에 약 30분 소요됩니다.

In [ ]:
# 분석 재생성 실행 여부
REGENERATE_ANALYSES = True  # 새 모델로 전체 재생성

if REGENERATE_ANALYSES:
    print('chamgab_analyses + price_factors 재생성 시작...')
    print('기존 데이터를 모두 삭제하고 새 모델로 재생성합니다.')
    print('예상 소요: ~30분')
    print('='*60)

    !cd /content/chamgab/ml-api && python -m scripts.batch_generate_analyses --regenerate --skip-sync

    print('\n재생성 완료!')
else:
    print('재생성 스킵 (REGENERATE_ANALYSES = False)')
    print('나중에 로컬에서 실행: python -m scripts.batch_generate_analyses --regenerate --skip-sync')

---
## 6. (선택) 재생성 결과 검증

In [ ]:
# 재생성 후 검증
if REGENERATE_ANALYSES:
    from supabase import create_client

    sb = create_client(os.environ['SUPABASE_URL'], os.environ['SUPABASE_SERVICE_KEY'])

    # 분석 건수
    analyses = sb.table('chamgab_analyses').select('id', count='exact').execute()
    factors = sb.table('price_factors').select('id', count='exact').execute()

    print(f'chamgab_analyses: {analyses.count}건')
    print(f'price_factors: {factors.count}건')

    # confidence 분포
    sample = sb.table('chamgab_analyses').select('confidence, chamgab_price').limit(1000).execute()
    import pandas as pd
    df = pd.DataFrame(sample.data)
    print(f'\nconfidence 분포:')
    print(f'  min: {df["confidence"].min()}')
    print(f'  median: {df["confidence"].median()}')
    print(f'  max: {df["confidence"].max()}')

    # 가격 비교 (최근 거래가 vs AI예측가)
    sample_with_price = sb.table('chamgab_analyses').select(
        'chamgab_price, property_id'
    ).limit(100).execute()

    print('\n검증 완료!')

---
## 사용 가이드

### Colab Secrets 설정 방법
1. 왼쪽 사이드바에서 열쇠 아이콘 클릭
2. `SUPABASE_URL` 추가 (값: `https://xxx.supabase.co`)
3. `SUPABASE_SERVICE_KEY` 추가 (값: `eyJ...`)
4. (선택) `HF_TOKEN` 추가 (HuggingFace 배포용)

### 월간 재훈련 루틴
1. 이 노트북을 Google Drive에 저장
2. 매월 1일 접속해서 **Run All** (Ctrl+F9)
3. 결과 확인 후 HuggingFace 배포

### PC 종료 시 유의사항
- 브라우저 닫아도 **최대 12시간** 실행 유지 (무료)
- Colab Pro: 최대 24시간
- 세션 만료 시 Google Drive에 저장된 모델은 유지됨